In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from utils import *
import pandas as pd

data = np.load('data.npz')
train_data = data['train_data']
test_data = data['test_data']
train_labels = data['train_labels']

train_feature1 = pd.read_csv('../dataset/feature1.csv')[:len(train_data)].values[:, 1:]
test_feature1 = pd.read_csv('../dataset/feature1.csv')[len(train_data):].values[:, 1:]

train_data.shape, train_labels.shape, test_data.shape, train_feature1.shape, test_feature1.shape

In [ ]:
train_data = tf.cast(train_data, tf.float32).numpy()
train_labels = tf.one_hot(train_labels, 20).numpy()

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
train_feature1 = ss.fit_transform(train_feature1)
test_feature1 = ss.transform(test_feature1)

In [ ]:
def res_block(seq, filters, kernal_size):
    cnn = keras.layers.Conv1D(filters, 1, padding='SAME', activation='relu')(seq)
    cnn = keras.layers.LayerNormalization()(cnn)

    cnn = keras.layers.Conv1D(filters, kernal_size, padding='SAME', activation='relu')(cnn)
    cnn = keras.layers.LayerNormalization()(cnn)

    cnn = keras.layers.Conv1D(filters, 1, padding='SAME', activation='relu')(cnn)
    cnn = keras.layers.LayerNormalization()(cnn)

    seq = keras.layers.Conv1D(filters, 1)(seq)
    seq = keras.layers.Add()([seq, cnn])
    return seq

def inception_block(seq, filters=128, kernal_size=5):
    seq = res_block(seq, filters, kernal_size)
    seq = keras.layers.MaxPooling1D(2)(seq)
    seq = keras.layers.SpatialDropout1D(0.3)(seq)
    seq = res_block(seq, filters//2, kernal_size)
    seq = keras.layers.GlobalAveragePooling1D()(seq)
    return seq

In [ ]:
def scheduler_func(epoch):
    if epoch < 25:
        rate = 1e-3
    elif epoch >= 25 and epoch < 35:
        rate = 3e-4
    elif epoch >= 35:
        rate = 9e-5
    return rate * 1.0

In [ ]:
def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape=input_shape[1:])

    input_ex = keras.layers.Input(shape=train_feature1.shape[1])
    seq_ex = input_ex

    seq_3 = inception_block(inputs, kernal_size=3)
    seq_5 = inception_block(inputs, kernal_size=5)
    seq_7 = inception_block(inputs, kernal_size=7)

    seq = keras.layers.concatenate([seq_3, seq_5, seq_7, seq_ex])
    seq = keras.layers.Dense(512, activation='relu')(seq)
    seq = keras.layers.Dropout(0.3)(seq)
    seq = keras.layers.Dense(128, activation='relu')(seq)
    seq = keras.layers.Dropout(0.3)(seq)
    outputs = keras.layers.Dense(num_classes, activation='softmax')(seq)

    model = keras.models.Model(inputs=[inputs, input_ex], outputs=[outputs])

    model.compile(optimizer=tf.optimizers.Adam(1e-3),
            loss=tf.losses.CategoricalCrossentropy(label_smoothing=0.1),           
            metrics=['accuracy'])

    return model

In [ ]:
kfcv_fit2(builder=lambda : build_model(train_data.shape, 20),
                x=[train_data, train_feature1], y=train_labels,
                epochs=40,
                checkpoint_path = './models/conv1d/',
                batch_size=64,
                extra_callbacks=[keras.callbacks.LearningRateScheduler(scheduler_func)],
                verbose=2,
                noise_std=0.075
                )

In [ ]:
np.save('conv1d_pred.npy', kfcv_predict('models/conv1d', [test_data, test_feature1]))